In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import re
import joblib
import string

In [2]:
fake_news = pd.read_csv('..\\Datasets\\Fake.csv')
fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [3]:
true_news = pd.read_csv('..\\Datasets\\True_news.csv')
true_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
fake_news['label'] = 0
true_news['label'] = 1

In [5]:
data = pd.concat([fake_news, true_news], axis=0)

In [6]:
data.sample(5)

,title,text,subject,date,label
9238,“NEVER-TRUMP” BUSTED! Former Jeb Bush Staffer ...,Just another member of the swamp who s about t...,politics,"Dec 9, 2017",0
20545,TRANSGENDER TARGET SUING Good Samaritan Who Sa...,The timing couldn t possibly be any worse ***W...,left-news,"May 20, 2016",0
20549,FURIOUS CUSTOMERS RESPOND To Home Depot Cashie...,The people who are angry about her wearing thi...,left-news,"May 20, 2016",0
1849,Paul Ryan Makes PERFECT Argument For Medicare...,Sometimes you just need to let people talk and...,News,"April 8, 2017",0
17909,"Globalization, migration fears reawaken German...",BERLIN (Reuters) - Most Germans are wary of an...,worldnews,"October 5, 2017",1


In [7]:
data =data.drop(['title','subject','date'], axis=1)


In [8]:
data.reset_index(inplace=True)

In [9]:
data.sample(5)

,index,text,label
43556,20075,MADRID (Reuters) - Catalonia will stop sending...,1
22145,22145,This is the final episode of Patrick LIVE on ...,0
27170,3689,"DUBAI (Reuters) - U.S. President Donald Trump,...",1
44329,20848,LUXEMBOURG/BRUSSELS (Reuters) - The EU s highe...,1
6051,6051,It would make sense why Trump claimed wages ar...,0


In [10]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [11]:
data['text'] = data['text'].apply(clean_text)

In [12]:
data.isnull().sum()

index    0
text     0
label    0
dtype: int64

In [13]:
# Remove rows with empty text
data = data[data['text'].str.strip() != '']
data.reset_index(drop=True, inplace=True)

In [14]:
data['text'] = data['text'].astype(str)

In [15]:
x=data['text']
y=data['label']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [16]:
vectorizer = TfidfVectorizer()
x_train_vectors = vectorizer.fit_transform(x_train)
x_test_vectors = vectorizer.transform(x_test)

In [17]:
lr = LogisticRegression()
lr.fit(x_train_vectors, y_train)

LogisticRegression()

In [18]:
predictions = lr.predict(x_test_vectors)
lr.score(x_test_vectors, y_test)

0.9846390168970814

In [19]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5727
           1       0.98      0.99      0.98      5340

    accuracy                           0.98     11067
   macro avg       0.98      0.98      0.98     11067
weighted avg       0.98      0.98      0.98     11067



In [20]:
joblib.dump(vectorizer, 'vectorizer.jb')
joblib.dump(lr, 'model.jb')

['model.jb']